# **Librerias usadas**

In [216]:
import pandas as pd
import numpy as np
from meteostat import Stations, Daily
from datetime import datetime
import csv
import os
import requests
import re
import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
import folium
import random

# **Web scraping**

## **Sección 1: Configuración e Infraestructura (Scraping)**

Este bloque se encarga de obtener la lista maestra de estaciones.

In [220]:
# --- FUNCIÓN DE EXTRACCIÓN (WEB SCRAPING) ---
def extraer_desde_html():
    """
    Escanea la web del SENAMHI para extraer coordenadas ocultas de 969 estaciones.
    """
    print("🌐 Iniciando Scraping de Infraestructura SENAMHI...")
    url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
    headers = {'User-Agent': 'Mozilla/5.0'}

    try:
        # 1. Petición a la web
        response = requests.get(url, headers=headers, timeout=10)
        contenido = response.text
        
        # 2. Extracción con Regex (Patrón: nombre, latitud, longitud)
        regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
        matches = re.findall(regex, contenido)
        
        if matches:
            print(f"✅ ¡ÉXITO! Se detectaron {len(matches)} estaciones activas.")
            # Guardamos en CSV para uso del sistema
            df = pd.DataFrame(matches, columns=["ESTACION", "LATITUD", "LONGITUD"])
            df["LATITUD"] = pd.to_numeric(df["LATITUD"])
            df["LONGITUD"] = pd.to_numeric(df["LONGITUD"])
            df.to_csv("SENAMHI_ESTACIONES_FINAL.csv", index=False)
            return df
        else:
            print("⚠️ No se encontraron patrones. Revisar estructura web.")
            return pd.DataFrame()

    except Exception as e:
        print(f"❌ Error de conexión: {e}")
        return pd.DataFrame()
        
if __name__ == "__main__":
    extraer_desde_html()

🌐 Iniciando Scraping de Infraestructura SENAMHI...
✅ ¡ÉXITO! Se detectaron 968 estaciones activas.


## **Sección 2: Lógica Geoespacial (Buscador de Vecinos)**

Este bloque recibe al avión y encuentra la estación física. Se ha fusionado la búsqueda con la validación.

In [223]:
# --- BUSCADOR GEODÉSICO Y AUDITORÍA ---
def asegurar_infraestructura_senamhi():
    """
    Verifica si tenemos el catálogo de estaciones. Si no, ejecuta el Scraping.
    """
    archivo = "SENAMHI_ESTACIONES_FINAL.csv"
    try:
        df = pd.read_csv(archivo)
        print(f"✅ Base de datos cargada: {len(df)} estaciones listas.")
        return df
    except FileNotFoundError:
        print("🌐 Iniciando Scraping de Infraestructura SENAMHI...")
        url = "https://www.senamhi.gob.pe/mapas/mapa-estaciones-2/"
        try:
            response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}, timeout=15)
            regex = r'"nom"\s*:\s*"(.*?)".*?"lat"\s*:\s*(-?\d+\.?\d*).*?"lon"\s*:\s*(-?\d+\.?\d*)'
            matches = re.findall(regex, response.text)
            
            if matches:
                df = pd.DataFrame(matches, columns=["ESTACION", "LATITUD", "LONGITUD"])
                df["LATITUD"] = pd.to_numeric(df["LATITUD"])
                df["LONGITUD"] = pd.to_numeric(df["LONGITUD"])
                df.to_csv(archivo, index=False)
                print(f"✅ ¡ÉXITO! Se detectaron y guardaron {len(df)} estaciones.")
                return df
        except Exception as e:
            print(f"❌ Error crítico en scraping: {e}")
            return pd.DataFrame()


def encontrar_y_validar_estacion(lat_obj, lon_obj, df_estaciones):
    """
    Busca la estación más cercana (Vecino Próximo) y audita la confianza.
    """
    # 1. Cálculo Vectorial de Distancia
    df_estaciones['distancia'] = np.sqrt(
        (df_estaciones['LATITUD'] - lat_obj)**2 + 
        (df_estaciones['LONGITUD'] - lon_obj)**2
    ) * 111.1  # Conversión aprox de grados a km
    
    # 2. Ganadora
    mejor = df_estaciones.sort_values('distancia').iloc[0]
    dist_km = round(mejor['distancia'], 2)
    
    # 3. Semáforo de Confianza
    if dist_km <= 5.0: confianza = "🟢 ALTA (Excelente)"
    elif dist_km <= 20.0: confianza = "🟡 MEDIA (Aceptable)"
    else: confianza = "🔴 BAJA (No Confiable)"
    
    return {
        "ESTACION": mejor['ESTACION'],
        "LAT": mejor['LATITUD'],
        "LON": mejor['LONGITUD'],
        "DIST_KM": dist_km,
        "CONFIANZA": confianza
    }

## **Sección 3: Inteligencia Climática (Meteostat + Feature Engineering)**

Este bloque descarga la historia real y calcula el riesgo operativo.

In [226]:
def obtener_inteligencia_climatica(lat, lon, inicio, fin):
    """
    Descarga historial real y calcula indicadores de riesgo operativo.
    """
    # 1. Descarga de Meteostat
    try:
        estaciones = Stations().nearby(lat, lon)
        estacion_id = estaciones.fetch(1).index[0]
        data = Daily(estacion_id, inicio, fin).fetch()
    except:
        return pd.DataFrame() # Retorna vacío si falla la API

    if data.empty: return pd.DataFrame()
    
    # 2. Feature Engineering (Cálculo de Indicadores)
    df = data.copy()
    # Limpieza
    df['wspd'] = df['wspd'].fillna(df['wspd'].median())
    df['pres'] = df['pres'].fillna(1013.0)
    df['prcp'] = df['prcp'].fillna(0)

    # Umbrales Dinámicos
    umbral_viento = df['wspd'].quantile(0.90) # Percentil 90
    
    # 3. Clasificador de Severidad (Pipeline Lógico)
    condiciones = []
    for _, row in df.iterrows():
        score = 0
        if row['wspd'] > umbral_viento: score += 1
        if row['prcp'] > 5.0: score += 1 # Lluvia fuerte
        
        # Detector Forense (Nieve/Aguanieve)
        if row['prcp'] > 0 and row['tmin'] <= 2.0:
            etiqueta = "❄️ RIESGO NIEVE"
        elif score >= 2:
            etiqueta = "🔴 SEVERA (Tormenta)"
        elif score == 1:
            etiqueta = "🟡 MODERADA"
        else:
            etiqueta = "🟢 NORMAL"
        condiciones.append(etiqueta)

    df['RIESGO_OPERATIVO'] = condiciones
    return df[['tmin', 'tmax', 'prcp', 'wspd', 'pres', 'RIESGO_OPERATIVO']]

## **Sección 4: Visualización**

Genera una auditoría forense visual mediante un mapa interactivo. Este bloque renderiza la ubicación de la aeronave y la estación SENAMHI validada, trazando el vector de distancia y el radio de confianza para certificar gráficamente la precisión de los datos meteorológicos utilizados.

In [228]:
def generar_mapa_auditoria(lat_vuelo, lon_vuelo, datos_estacion):
    print("🎨 Generando mapa de evidencia forense...")
    m = folium.Map(location=[lat_vuelo, lon_vuelo], zoom_start=13)
    
    # Marcadores
    folium.Marker([lat_vuelo, lon_vuelo], popup="AERONAVE", icon=folium.Icon(color="red", icon="plane", prefix="fa")).add_to(m)
    folium.Marker([datos_estacion['LAT'], datos_estacion['LON']], popup=f"SENAMHI: {datos_estacion['ESTACION']}", icon=folium.Icon(color="green", icon="cloud", prefix="fa")).add_to(m)
    
    # Línea de conexión y Radio de Confianza
    folium.PolyLine([[lat_vuelo, lon_vuelo], [datos_estacion['LAT'], datos_estacion['LON']]], color="blue", dash_array='5, 10').add_to(m)
    folium.Circle([datos_estacion['LAT'], datos_estacion['LON']], radius=5000, color="green", fill=True, fill_opacity=0.1, popup="Radio Confianza 5km").add_to(m)
    
    m.save("MAPA_AUDITORIA_FINAL.html")
    print("✅ Mapa guardado: MAPA_AUDITORIA_FINAL.html")

## **¿Qué tiempo hace en esas estaciones y qué tan peligroso es para volar?**

## **Sección 5: Ejecución Maestra (Merge + Análisis de datos)**

Este es el único bloque que necesitas ejecutar al final para demostrar que todo funciona junto.

In [280]:
# ==============================================================================
# SECCIÓN 5: EJECUCIÓN MAESTRA (PIPELINE + EDA COMPLETO)
# ==============================================================================
if __name__ == "__main__":
    print("\n🚀 INICIANDO PIPELINE DE INTEGRACIÓN AERO-METEOROLÓGICA...\n")

    # 1. INPUT (Simulación de Vuelo en Aproximación)
    lat_vuelo, lon_vuelo = -12.02, -77.11 
    print(f"📍 Aeronave detectada en coordenadas: {lat_vuelo}, {lon_vuelo}")

    # ---------------------------------------------------------
    # FASE 1: INFRAESTRUCTURA (Scraping)
    # ---------------------------------------------------------
    df_infra = asegurar_infraestructura_senamhi()

    if not df_infra.empty:
        # ---------------------------------------------------------
        # FASE 2: LÓGICA ESPACIAL (Buscador Geodésico)
        # ---------------------------------------------------------
        resultado_geo = encontrar_y_validar_estacion(lat_vuelo, lon_vuelo, df_infra)
        print(f"\n📡 Estación Vinculada: {resultado_geo['ESTACION']}")
        print(f"📏 Distancia al objetivo: {resultado_geo['DIST_KM']} km")
        print(f"🚦 Auditoría de Confianza: {resultado_geo['CONFIANZA']}")

        # ---------------------------------------------------------
        # FASE 3: INTELIGENCIA (Meteostat Real + Feature Engineering)
        # ---------------------------------------------------------
        print("\n⚡ Descargando y Analizando Historial Operativo (2024)...")
        # Esta función ya limpia y clasifica internamente
        df_riesgo = obtener_inteligencia_climatica(
            resultado_geo['LAT'], 
            resultado_geo['LON'], 
            datetime(2024, 1, 1), 
            datetime(2024, 12, 31)
        )

        if not df_riesgo.empty:
            # ---------------------------------------------------------
            # FASE 4: ANÁLISIS EXPLORATORIO DE DATOS (EDA COMPLETO)
            # ---------------------------------------------------------
            print("\n" + "="*50)
            print("📊 REPORTE TÉCNICO Y ANÁLISIS EXPLORATORIO (EDA)")
            print("="*50)

            print("\n1️⃣ VISTA PREVIA DE DATOS PROCESADOS (Head):")
            print(df_riesgo.head().to_string())

            print("\n2️⃣ ESTRUCTURA DEL DATASET (Info):")
            df_riesgo.info() # Se imprime directo en consola

            print("\n3️⃣ ESTADÍSTICAS DESCRIPTIVAS (Describe):")
            print(df_riesgo.describe().to_string())

            print("\n4️⃣ DISTRIBUCIÓN DE RIESGO OPERATIVO (Conteo):")
            print(df_riesgo['RIESGO_OPERATIVO'].value_counts().to_string())

            # ---------------------------------------------------------
            # FASE 5: ENTREGABLES FINALES (CSV + MAPA)
            # ---------------------------------------------------------
            print("\n" + "="*50)
            print("📦 GENERANDO ENTREGABLES FINALES")
            print("="*50)
            
            print("📋 Muestra de Clasificación Final (Últimos 5 días):")
            print(df_riesgo.tail(5)[['wspd', 'prcp', 'RIESGO_OPERATIVO']].to_string())
            
            # Guardar CSV
            df_riesgo.to_csv("senamhi_clima_indicadores.csv")
            print("\n💾 Archivo CSV guardado: 'senamhi_clima_indicadores.csv'")
            
            # Generar Mapa
            generar_mapa_auditoria(lat_vuelo, lon_vuelo, resultado_geo)
            
        else:
            print("❌ Error: No se encontraron datos climáticos en Meteostat para esta ubicación.")
    else:
        print("❌ Fallo crítico en infraestructura: No se pudo cargar la base de datos.")


🚀 INICIANDO PIPELINE DE INTEGRACIÓN AERO-METEOROLÓGICA...

📍 Aeronave detectada en coordenadas: -12.02, -77.11
✅ Base de datos cargada: 968 estaciones listas.

📡 Estación Vinculada: SAN MARTIN DE PORRES
📏 Distancia al objetivo: 3.09 km
🚦 Auditoría de Confianza: 🟢 ALTA (Excelente)

⚡ Descargando y Analizando Historial Operativo (2024)...

📊 REPORTE TÉCNICO Y ANÁLISIS EXPLORATORIO (EDA)

1️⃣ VISTA PREVIA DE DATOS PROCESADOS (Head):
            tmin  tmax  prcp  wspd    pres RIESGO_OPERATIVO
time                                                       
2024-01-01  22.0  25.2   1.0  19.1  1014.0         🟢 NORMAL
2024-01-02  22.0  26.0   0.0  20.9  1013.4       🟡 MODERADA
2024-01-03  22.0  25.3   0.0  21.5  1013.5       🟡 MODERADA
2024-01-04  21.8  26.0   0.0  16.4  1014.1         🟢 NORMAL
2024-01-05  22.0  26.0   0.0  24.7  1013.2       🟡 MODERADA

2️⃣ ESTRUCTURA DEL DATASET (Info):
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 366 entries, 2024-01-01 to 2024-12-31
Data columns (tota